[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=examples/PySDM_examples/Shipway_and_Hill_2012/fig_1.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Shipway_and_Hill_2012/fig_1.ipynb)

#### based on Fig. 1 from Shipway & Hill 2012 (Q. J. Royal Meteo. Soc. 138)  "_Diagnosis of systematic differences between multiple parametrizations of warm rain microphysics using a kinematic framework_"   
https://doi.org/10.1002/qj.1913

**NOTES**: 
- constant momentum profile rather than constant velocity profile is used herein
- enabling precipitation interpretted as turning on sedimentation and collisions
- pressure at z=0 not given in the paper, assumed (see settings.py)
- domain extended below z=0 to mimic particle inflow

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [ ]:
from PySDM_examples.Shipway_and_Hill_2012 import Settings, Simulation, plot
from PySDM.physics import si
from PySDM.exporters import NetCDFExporter_1d

In [ ]:
common_params = {
    "n_sd_per_gridbox": 256,
    "dt": 5 * si.s,
    "dz": 50 * si.m,
    "p0": 990 * si.hPa,
    "kappa": .9,
    "particles_per_volume_STP": 50 / si.cm**3
}

output = {}
settings = {}
simulation = {}
for rho_times_w in (
    2 * si.kg/si.m**3 * si.m/si.s,
    3 * si.kg/si.m**3 * si.m/si.s
):
    for precip in (
        False, 
        True
    ):
        key = f"rhow={rho_times_w}_p={precip}"
        settings[key] = Settings(
            **common_params,
            rho_times_w_1=rho_times_w,
            precip=precip
        )
        simulation[key] = Simulation(settings[key])
        output[key] = simulation[key].run().products


In [ ]:
for p in (False, True):
    for rho_times_w in (2, 3):
        plot(var='qc', qlabel='$q_c$ [g/kg]', fname=f'qc_rhow={rho_times_w}_p={p}.pdf',
             output=output[f'rhow={rho_times_w}.0_p={p}'])

In [ ]:
p = True
for rho_times_w in (2, 3):
    plot(var='qr', qlabel='$q_r$ [g/kg]', fname=f'qr_rhow={rho_times_w}_p={p}.pdf',
         output=output[f'rhow={rho_times_w}.0_p={p}'])

In [ ]:
for p in (False, True):
    for rho_times_w in (2, 3):
        key = f"rhow={rho_times_w}.0_p={p}"
        filename = 'products_' + key + '.nc'
        nc_exporter = NetCDFExporter_1d(output[key], settings[key], simulation[key], filename)
        nc_exporter.run()